# Pre-procesing 

In [1]:
from random import choice
from collections import defaultdict
from datetime import datetime
from pathlib import Path
from shutil import rmtree
import pandas as pd
import json
import dateutil.parser
import os
from urllib.parse import quote

In [2]:
#Datos de kaggle
RAW_DATA = Path("raw_dataset").absolute()

#Output de datos (datos limpios)

CLEAN_DATA = Path("clean_dataset").absolute()

if  CLEAN_DATA.exists():
    rmtree(CLEAN_DATA)
    os.mkdir(CLEAN_DATA)
else:
    os.mkdir(CLEAN_DATA)

FRAC = 0.05

## Loading the data

In [3]:

countries = ["CA","DE","FR","GB","JP","KR","IN","MX","RU","US"] #Lista de países.
videos = {} #Lista con los dataframes de archivos csv
categories = {} # Lista con los diccionarios de los archivos json.
print("Abriendo archivos del dataset-->")
for i,country in enumerate(countries):
    print(f"\tPais: {country}  {i+1}/{len(countries)} ",end = "\r")
    file_csv = RAW_DATA.joinpath(f"{country}videos.csv")
    videos[country] = pd.read_csv(file_csv,encoding="ISO-8859-1",lineterminator="\n")
    videos[country].columns = [x.strip() for x in videos[country].columns]


    with open(RAW_DATA.joinpath(f"{country}_category_id.json")) as file:
        items = json.load(file)["items"]
        temp = {int(x["id"]): x for x in items} #La llave de cada categoria es su id.
        val = defaultdict(lambda: "NULL")
        val["snippet"] = defaultdict(lambda:"NULL")
        val["snippet"]["title"] = 'No categoria'
        categories[country] = defaultdict(lambda : val,temp)
print("\nDatos Cargados!")

Abriendo archivos del dataset-->
	Pais: US  10/10 
Datos Cargados!


In [4]:
# veamos el numero de nulos sin contar la descripcion de cada DF
print("Numero de filas con algun nulo (sin contar descripcion) en el DF de ")
for key,df_clean in videos.items():
    df = df_clean.copy().drop(["description"],axis = 1)
    df1 = df[df.isna().any(axis=1)]
    print(f"\tPais --> {key}: {len(df1)}")


Numero de filas con algun nulo (sin contar descripcion) en el DF de 
	Pais --> CA: 0
	Pais --> DE: 0
	Pais --> FR: 0
	Pais --> GB: 0
	Pais --> JP: 0
	Pais --> KR: 0
	Pais --> IN: 0
	Pais --> MX: 0
	Pais --> RU: 0
	Pais --> US: 0


In [5]:
f = '%Y-%m-%dT%H:%M:%S'

def get_category_name(series,key):
    return series.apply(lambda x: categories[key][x]["snippet"]["title"])

def get_publish_timestamp(series):
    return series.apply(lambda x: dateutil.parser.isoparse(x).strftime(f))

def get_trending_timestamp(series):
    return series.apply(lambda x: datetime.strptime(x,"%y.%d.%m").strftime(f))

In [6]:
for key,df in videos.items():
    df["category"] = get_category_name(df["category_id"],key) # Agregamos el nombre de la categoria
    # Parseamos las fechas para llegar y comparar como timestamps
    df["publish_timestamp"] = get_publish_timestamp(df["publish_time"]) 
    df["trending_timestamp"] = get_trending_timestamp(df["trending_date"]) 
    df.drop(["video_id","category_id","thumbnail_link"],axis =1,inplace=True)

In [7]:
print("Escribiendo nuevo dataset-->")
df_all = pd.DataFrame(columns = ["country"]+ list(videos["CA"].columns))
for i,(key,df) in enumerate(videos.items()):
    df["country"] = key
    print(f"Escribiendo {key}videos.csv   {i+1}/{len(videos.items())}",end = "\r")
    # get 10% sample of df
    # remove rows with no tags
    df = df[df["tags"] != "[none]"]
    df = df.sample(frac=FRAC)
    df_all = pd.concat([df_all,df],ignore_index = True)
# Add numeric id to each row called video_id
df_all["video_id"] = df_all.index

print("\n Dataset listo!")

Escribiendo nuevo dataset-->
Escribiendo USvideos.csv   10/10
 Dataset listo!


In [8]:
df_all.head()

,country,trending_date,title,channel_title,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,category,publish_timestamp,trending_timestamp,video_id
0,CA,18.07.06,Can you solve the penniless pilgrim riddle? - ...,TED-Ed,2018-06-04T15:03:15.000Z,"TED|""TED-Ed""|""TED Ed""|""Teded""|""Ted Education""|...",494015,17673,367,1240,False,False,False,"If you enjoy our riddles, we highly recommend ...",Education,2018-06-04T15:03:15,2018-06-07T00:00:00,0
1,CA,18.26.01,FLOWER BOY: a conversation,"Tyler, The Creator",2018-01-22T15:33:36.000Z,"tyler the creator|""jerrod carmichael""|""scum fu...",911172,62082,408,4632,False,False,False,Jerrod Carmichael / Tyler Okonma\n\nDirected B...,Education,2018-01-22T15:33:36,2018-01-26T00:00:00,1
2,CA,18.04.02,Comment Awards v55,Comment Awards,2018-02-03T23:18:41.000Z,"comment awards|""comment""|""awards""|""dank memes ...",322991,10093,279,2394,False,False,False,"Welcome to Comment Awards, your #1 source for ...",Comedy,2018-02-03T23:18:41,2018-02-04T00:00:00,2
3,CA,18.21.05,Heartwarming Moment Doria Holds Hands with Pri...,MIU MIU,2018-05-20T10:27:20.000Z,"Doria|""MEGHAN MARKLE'S MOTHER""|""meghan's mothe...",606359,2774,408,389,False,False,False,Heartwarming Moment Doria Holds Hands with Pri...,People & Blogs,2018-05-20T10:27:20,2018-05-21T00:00:00,3
4,CA,18.08.05,Jada Pinkett Smith's Mom Had Jada When She Was...,The Real Daytime,2018-05-07T17:49:39.000Z,"the real|""daytime""|""talk show""|""women""|""tamera...",69361,1204,15,76,False,False,False,NaN,Entertainment,2018-05-07T17:49:39,2018-05-08T00:00:00,4


# New Tables for tags

In [9]:
tags_series = df_all["tags"].apply(lambda x: x.split("|"))
tags_series = tags_series.apply(lambda x: [y.strip() for y in x])
tags = [item for sublist in tags_series for item in sublist] # Lista con todas las tags.
new_tags = []
for tag in tags:
    # remove " from tags
    tag = tag.replace('"',"")
    # Replace spaces with _
    tag = tag.replace(" ","_")
    tag = quote(tag,safe='/:?=&')
    new_tags.append(tag)

tags = new_tags


# Count frequency of each tag
freq_dict = {}
for tag in tags:
    if tag in freq_dict:
        freq_dict[tag] += 1
    else:
        freq_dict[tag] = 1

# Remove tags that appear less than 100 times

thresh = 100

freq_dict = {key:value for key,value in freq_dict.items() if value >= thresh}
tag_set = freq_dict.keys()


# Remove from every elemnt in tag series values that are not in final_set
def remove_tags(x):
    res = []
    for elem in x:
        elem  = elem.replace('"',"")
        # Replace spaces with _
        elem = elem.replace(" ","_")
        elem = quote(elem,safe='/:?=&')
        if elem in tag_set:
            res.append(elem)

    return res

tags_series = tags_series.apply(remove_tags)



# print info

print("\n------------BEFORE------------\n")
print(f"Number of unique tags : {len(set(new_tags))}")
print(f"Total number of tags: {len(new_tags)}")
print("\n------------AFTER------------\n")

print(f"Number of unique tags : {len(tag_set)}")
print("Total number of tags.",sum(freq_dict.values()))
print("Rows without tags {}".format(len(tags_series[tags_series.apply(lambda x: len(x) == 0)])))



------------BEFORE------------

Number of unique tags : 151086
Total number of tags: 320725

------------AFTER------------

Number of unique tags : 67
Total number of tags. 11604
Rows without tags 11507


In [10]:

# create enw dataframe with 2 columns video_id and tag_id
video_tag_df = pd.DataFrame(columns = ["video_id","tag_name"],index = range(sum(freq_dict.values())))

In [11]:
counter = 0
for elem in enumerate(tags_series):
    L = elem[1]
    for tag in L:
        video_tag_df.loc[counter] = [elem[0],tag]
        counter += 1


In [12]:
df_all = df_all.drop(["tags","description"],axis = 1)

In [13]:
video_tag_df

,video_id,tag_name
0,1,interview
1,2,funny
2,4,talk_show
3,7,youtube
4,8,politics
...,...,...
11599,16906,Music
11600,16906,Records
11601,16910,Trailer
11602,16910,television


In [14]:
# show uniqute values of category
print("Unique values of category")



# result: http://www.oschina.net/search?scope=bbs&q=C%E8%AF%AD%E8%A8%80
quote('http://www.oschina.net/search?scope=bbs&q=C语言',safe='/:?=&')
# replace " " with "_"
df_all["category"] = df_all["category"].apply(lambda x: x.replace(" ","_"))
# remove control caracters from channel_tittle
df_all["channel_title"] = df_all["channel_title"].apply(lambda x: x.replace(" ","_"))
df_all["channel_title"] = df_all["channel_title"].apply(lambda x: quote(x,safe='/:?=&'))

print(df_all["category"].unique())

Unique values of category
['Education' 'Comedy' 'People_&_Blogs' 'Entertainment' 'Sports'
 'News_&_Politics' 'Autos_&_Vehicles' 'Music' 'Gaming' 'Movies'
 'Science_&_Technology' 'Howto_&_Style' 'Film_&_Animation'
 'Pets_&_Animals' 'Shows' 'No_categoria' 'Travel_&_Events'
 'Nonprofits_&_Activism']


## Saving the new dataset

In [15]:
# Save dataframes to csv
df_all.to_csv(CLEAN_DATA /"videos.csv",index = False)
video_tag_df.to_csv(CLEAN_DATA /"video_tags.csv",index = False)